# NIRCam imager simulator

In [1]:
import os

# For examining outputs
from glob import glob
from scipy.stats import sigmaclip
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
os.environ["MIRAGE_DATA"] = "/Users/spacegrey/opt/anaconda3/envs/mirage/lib/python3.7/site-packages/mirage/reference_files/mirage_data/"
os.environ["CRDS_DATA"] = "/user/myself/crds_cache"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

In [4]:
# mirage imports
import yaml
from mirage import imaging_simulator
from mirage.seed_image import catalog_seed_image
from mirage.dark import dark_prep
from mirage.ramp_generator import obs_generator
from mirage.yaml import yaml_generator

from mirage.catalogs.catalog_generator import ExtendedCatalog
from mirage.catalogs.create_catalog import combine_catalogs
from mirage.imaging_simulator import ImgSim
from mirage.reference_files.downloader import download_file
#from mirage.seed_image.fits_seed_image import ImgSeed

## Generating input yaml files

In [5]:
# Specify the xmnl and pointing files exported from APT
xml_file = './image2_data/image2.xml'
pointing_file = './image2_data/image2.pointing'

In [28]:
# Source catalogs to be used.
cat_dict = {'extended':'image2_data/source_catalog_file.cat'}

# Set reference file values. 
# Setting to 'crds_full_name' will search for and download needed calibration reference files (commonly referred to as CRDS reference files) when the yaml_generator is run. 
# Setting to 'crds' will put placeholders in the yaml files and save the downloading for when the simulated images are created.
reffile_defaults = 'crds_full_name'

# Optionally set the cosmic ray library and rate
cosmic_rays = {'library': 'SUNMAX', 'scale': 1.0}

# Optionally set the background signal rates to be used
background = 'medium'

# Optionally set the telescope roll angle (PAV3) for the observations
pav3 = 0

# Optionally set the observation date to use for the data. Note that this information
# is placed in the headers of the output files, but not used by Mirage in any way.
dates = '2022-10-31'

ghosts = False
convolve_ghosts = False



In [29]:
# Set the directory into which the yaml files and simulation output will be written
yaml_dir = './image2_data/'
simulation_dir = './image2_sim/'

# specify the data reduction state of the Mirage outputs
datatype = 'linear'

In [33]:
# Run the yaml generator
yam = yaml_generator.SimInput(input_xml=xml_file, pointing_file=pointing_file,
                              catalogs=cat_dict, cosmic_rays=cosmic_rays,
                              background=background, roll_angle=pav3,
                              dates=dates, reffile_defaults=reffile_defaults,
                              add_ghosts=ghosts, convolve_ghosts_with_psf=convolve_ghosts,
                              verbose=True, output_dir=yaml_dir,
                              simdata_output_dir=simulation_dir,
                              datatype=datatype)
yam.create_inputs()

ds.stsci.edu/unchecked_get/references/jwst/jwst_nircam_distortion_0095.asdf' : HTTP Error 503: Service Temporarily Unavailable
CRDS - INFO -  Fetching  /Users/spacegrey/crds_cache/references/jwst/nircam/jwst_nircam_gain_0049.fits   16.8 M bytes  (3 / 10 files) (0 / 352.5 M bytes)
CRDS - ERROR -  Failure downloading file 'jwst_nircam_gain_0049.fits' : Error fetching data for 'jwst_nircam_gain_0049.fits' at CRDS server 'https://jwst-crds.stsci.edu' with mode 'http' : Failed downloading 'jwst_nircam_gain_0049.fits' from url 'https://jwst-crds.stsci.edu/unchecked_get/references/jwst/jwst_nircam_gain_0049.fits' : HTTP Error 503: Service Temporarily Unavailable
CRDS - INFO -  Fetching  /Users/spacegrey/crds_cache/references/jwst/nircam/jwst_nircam_ipc_0024.fits   11.5 K bytes  (4 / 10 files) (0 / 352.5 M bytes)
CRDS - ERROR -  Failure downloading file 'jwst_nircam_ipc_0024.fits' : Error fetching data for 'jwst_nircam_ipc_0024.fits' at CRDS server 'https://jwst-crds.stsci.edu' with mode 'http

In [26]:
yfiles = glob(os.path.join(yaml_dir,'jw*.yaml'))

In [43]:
# Run all steps of the imaging simulator for yaml file #1
yfile = "image2_data/jw00005001001_01101_00001_nrca1.yaml"

img_sim = imaging_simulator.ImgSim()
img_sim.paramfile = yfile
img_sim.create()

2021-05-09 20:57:08,543 - mirage.imaging_simulator - INFO - 

Running imaging_simulator....

2021-05-09 20:57:08,543 - stpipe - INFO - 

Running imaging_simulator....

2021-05-09 20:57:08,543 - stpipe - INFO - 

Running imaging_simulator....

2021-05-09 20:57:08,547 - mirage.imaging_simulator - INFO - using parameter file: image2_data/jw00005001001_01101_00001_nrca1.yaml
2021-05-09 20:57:08,547 - stpipe - INFO - using parameter file: image2_data/jw00005001001_01101_00001_nrca1.yaml
2021-05-09 20:57:08,547 - stpipe - INFO - using parameter file: image2_data/jw00005001001_01101_00001_nrca1.yaml
2021-05-09 20:57:08,587 - mirage.seed_image.catalog_seed_image - INFO - 

Running catalog_seed_image..

2021-05-09 20:57:08,587 - stpipe - INFO - 

Running catalog_seed_image..

2021-05-09 20:57:08,587 - stpipe - INFO - 

Running catalog_seed_image..

2021-05-09 20:57:08,590 - mirage.seed_image.catalog_seed_image - INFO - Reading parameter file: image2_data/jw00005001001_01101_00001_nrca1.yaml

20

OSError: WARNING: unable to read in None

In [44]:
with open(yfile) as file_obj:
        params = yaml.safe_load(file_obj)
filename = params['Reffiles']['transmission']
filename is not None

False

## Import fits image

In [37]:
fitsfile = "MIRI_F1000W.fits"
# Choose one of the yaml files just created
yfile = 'image2_data/jw00005001001_01101_00001_nrca1.yaml'

with open(yfile) as file_obj:
        params = yaml.safe_load(file_obj)

ra = params['Telescope']['ra']
dec = params['Telescope']['dec']
pav3 = params['Telescope']['rotation']

# Define the output files and directories
sim_data_dir = params['Output']['directory']
simulated_filename = params['Output']['file']
crop_file = simulated_filename.replace('.fits', '_cropped_from_image.fits')
crop_file = os.path.join(sim_data_dir, crop_file)
blot_file = simulated_filename.replace('.fits', '_blotted_seed_image.fits')


In [82]:
# Crop from the mosaic and resample for the desired detector/aperture
mosaic_fwhm = 0.062  # arcseconds
seed = ImgSeed(paramfile=yfile, mosaic_file=fitsfile, cropped_file=crop_file,
                outdir=sim_data_dir, blotted_file=blot_file, psf_file=None,
                 mosaic_fwhm=mosaic_fwhm, mosaic_fwhm_units='arcsec', gaussian_psf=False)
seed.crop_and_blot()

2021-05-04 20:58:34,227 - mirage.seed_image.fits_seed_image - INFO - 
2021-05-04 20:58:34,230 - mirage.seed_image.fits_seed_image - INFO - 
2021-05-04 20:58:34,230 - mirage.seed_image.fits_seed_image - INFO - Running fits_seed_image....
2021-05-04 20:58:34,231 - mirage.seed_image.fits_seed_image - INFO - 
2021-05-04 20:58:34,232 - mirage.seed_image.fits_seed_image - INFO - using parameter file: image2_data/jw00005001001_01101_00001_nrca1.yaml
2021-05-04 20:58:34,234 - mirage.seed_image.fits_seed_image - INFO - 
2021-05-04 20:58:34,238 - mirage.seed_image.fits_seed_image - INFO - Original log file name: ./mirage_latest.log


TypeError: bad operand type for abs(): 'NoneType'